In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [15]:
import collections
import glob
import json

from multiprocessing.pool import Pool, ThreadPool

import numpy as np
import pandas as pd

import dask
import dask.dataframe as dd
import dask.bag as db
from dask.diagnostics import ProgressBar, Profiler, ResourceProfiler, CacheProfiler

In [4]:
import random_indexing as ri

In [5]:
def load(f_name):
    import csv
    
    with open(f_name) as f:
        return list(r['text'] or '' for r in csv.DictReader(f))

In [8]:
f_names = sorted(glob.glob('tweet_dump.csv'))

In [30]:
%%time
_ = (
    db.read_text('rehydrated_tweets_dump.csv')
    .map(lambda r: print(r) or json.loads)
    .take(13, compute=False)
    .pluck('text', '')
    .flatten().flatten().frequencies()
    .to_dataframe(meta=[('letter', str), ('count', int)])
    .set_index('letter')
)

with ThreadPool(processes=2) as pool:
    with ProgressBar():
        with dask.set_options(pool=pool, get=dask.threaded.get):
            vocabulary = _.compute()['count']

vocabulary[''] = 0

vocabulary = vocabulary.sort_values(ascending=False)

{"tweet_id":853035540559671296,"text":"Just posted a photo @ Riga, Latvia https://t.co/NmtoSkTRtN","language":"en","screen_names":[16],"hashtags":[],"is_retweet":null}

{"tweet_id":853036662594707456,"text":"RT @DelfiLV: Vienkārši un eleganti: kā no zeķes izveidot mīlīgu zaķīti https://t.co/67vFBXdWVo https://t.co/3T4kN8n2HJ","language":"lv","screen_names":[112],"hashtags":[],"is_retweet":1}

{"tweet_id":853038492380794880,"text":"Mission @aigrettefoto #badgirlzzz #mission #SaveTheWorld #instagram #photo #iPhonephotography #secretmission #Latvia #Riga https://t.co/6mcFfqAevW","language":"en","screen_names":[3782],"hashtags":["badgirlzzz", "instagram", "iphonephotography", "latvia", "mission", "photo", "riga", "savetheworld", "secretmission"],"is_retweet":null}

{"tweet_id":853040176620326914,"text":"Foto: Kā noveco supermodeļu 'lielais sešinieks' https://t.co/4mO1jxUudq https://t.co/aA7RLhzItG","language":"lv","screen_names":["@@DELFI", "@@MEDIA", "@delfilv"],"hashtags":[],"is_retweet"

In [19]:
vocabulary

letter
     95
t    65
i    54
s    48
o    42
e    41
/    33
n    33
a    33
h    22
p    21
c    19
m    18
.    17
l    17
r    16
v    16
d    16
:    16
u    14
#    14
g    13
z    13
k    12
ī    10
ā     8
R     8
О     7
f     6
V     6
     ..
Л     2
Y     2
д     2
г     1
с     1
🍷     1
Ā     1
+     1
ь     1
-     1
ч     1
у     1
x     1
ņ     1
3     1
Д     1
X     1
Z     1
🔥     1
9     1
р     1
м     1
М     1
B     1
П     1
M     1
Я     1
G     1
H     1
      0
Name: count, Length: 107, dtype: int64

In [14]:
dask.set_options?

In [ ]:
len(vocabulary)

In [ ]:
D = 10_000

In [ ]:
features = ri.Features(vocabulary.index, D=D)

In [ ]:
def pool_init():
    import mp

In [ ]:
%%time

def vectorize(t):
    import mp
    mp.features
    return mp.features
    
_ = dd.read_csv('tweet_dump/0000*')['text'].repartition(npartitions=4).map(vectorize)

with Pool(processes=2, initializer=pool_init) as pool:
    with dask.set_options(pool=pool, get=dask.multiprocessing.get), ProgressBar():
            result = _.compute()

In [ ]:
result.shape

In [ ]:
import os

In [ ]:
os.getpid()

In [ ]:
(
    dd.read_csva
    (
        'tweet_dump/0000',
        usecols=['tweet_id', 'text']
    )
    .set_index('tweet_id')
    ['text'].map(list)
    .concat()
#     .sum(skipna=True)
#     .nlargest(10)
).head(10)

In [ ]:
(
    pd.read_csv(
        'tweet_dump/0000',
        usecols=['tweet_id', 'text']
    )
    .set_index('tweet_id')
    ['text'].apply(lambda r: pd.Series(list(r)).value_counts())
    .sum(skipna=True)
    .nlargest(10)
)